In [342]:
import os
import pandas as pd
import numpy as np

In [343]:
def load_data():
    df = pd.read_csv('weather_data.csv')
    df['time1'] = df['date'] + ' ' + df['time']
    df = df.drop(columns={'Unnamed: 0', 'date', 'time'})
    df['time1'] =  pd.to_datetime(df['time1'], format='%m/%d/%Y %H:%M')
    df.rename(columns={ 'time1': "time" }, inplace = True)
    df = df.set_index('time')

    # drop dublicates
    df = df.loc[~df.index.duplicated(keep='first')]
    return df

In [344]:
def replace_nan(df):
    # replace '-----' to nan
    df['T'].replace('-----', np.nan, inplace=True)

    # df['Tmax'].replace('-----', np.nan, inplace=True)
    # df['Tmin'].replace('-----', np.nan, inplace=True)
    # df['Td'].replace('-----', np.nan, inplace=True)

    df['T'] = df['T'].astype(float)
    # df['Tmax'] = df['Tmax'].astype(float)
    # df['Tmin'] = df['Tmin'].astype(float)
    # df['Td'] = df['Td'].astype(float)
    
    df['T'] = df['T'].where(df['T'].values == 999, other=(df['T'].fillna(method='ffill') + df['T'].fillna(method='bfill'))/2)
    df[df['T'].isnull()]
    
    return df

In [345]:
def shift_rows(df):
    # dfao = dfao.append(pd.Series(name=datetime.datetime(2020, 5, 30, 9, 0)))
    # dfao = dfao.append(pd.Series(name=datetime.datetime(2020, 5, 30, 12, 0)))
    
    import datetime
    datetime_3 = df.iloc[-1:].index + datetime.timedelta(hours=3)
    datetime_6 = df.iloc[-1:].index + datetime.timedelta(hours=6)
    
    df = df.append(pd.DataFrame(index=datetime_3))
    df = df.append(pd.DataFrame(index=datetime_6))

    # shift for utc to omsk time change !!!
    df = df.shift(periods=2)
    df = df.iloc[2:]
    df.tail(5)
    return df

In [346]:
# Serbakul
folder_city = 'Serbakul' # Sargatskoe # Serbakul # Kalacinsk
os.chdir("B:/Projects/CourseWork/coursework/triangle_ogimet/data/" + folder_city)
print(os.getcwd())

B:\Projects\CourseWork\coursework\triangle_ogimet\data\Serbakul


In [347]:
df = load_data()
df = replace_nan(df)
df = shift_rows(df)
df_ser = df.copy()

In [348]:
# Sargatskoe
folder_city = 'Sargatskoe' # Sargatskoe # Serbakul # Kalacinsk
os.chdir("B:/Projects/CourseWork/coursework/triangle_ogimet/data/" + folder_city)
print(os.getcwd())

B:\Projects\CourseWork\coursework\triangle_ogimet\data\Sargatskoe


In [349]:
df = load_data()
df = replace_nan(df)
df = shift_rows(df)
df_sar = df.copy()

In [350]:
# Kalacinsk
folder_city = 'Kalacinsk' # Sargatskoe # Serbakul # Kalacinsk
os.chdir("B:/Projects/CourseWork/coursework/triangle_ogimet/data/" + folder_city)
print(os.getcwd())

B:\Projects\CourseWork\coursework\triangle_ogimet\data\Kalacinsk


In [351]:
df = load_data()
df = replace_nan(df)
df = shift_rows(df)
df_kal = df.copy()

In [352]:
df_three = df_kal.copy()
df_three.rename(columns={"T": "kal"}, inplace=True)
df_sar = pd.DataFrame(df_sar['T'])
df_ser = pd.DataFrame(df_ser['T'])

df_sar.rename(columns={"T": "sar"}, inplace=True)
df_ser.rename(columns={"T": "ser"}, inplace=True)

# df_three = df_three.append(df_sar)
# df_three = df_three.append(df_ser)

df_three.drop(columns={'Td', 'Tmax', 'Tmin', 'ddd', 'ff', 'prec', 'nt', 'nh', 'h'}, inplace=True)
# df_three['mean'] = df_three.mean(axis=1)
df_three

,kal
time,
1999-09-15 00:00:00,14.1
1999-09-15 03:00:00,12.6
1999-09-16 18:00:00,11.5
1999-09-16 21:00:00,12.5
1999-09-17 00:00:00,12.0
...,...
2018-07-25 03:00:00,13.0
2018-07-25 06:00:00,11.8
2018-07-25 09:00:00,17.1


In [353]:
df_three = df_three.merge(df_sar, how='inner', right_index=True, left_index=True)
df_three

,kal,sar
time,,
1999-09-17 03:00:00,10.1,16.9
1999-09-17 06:00:00,7.9,13.0
1999-09-17 12:00:00,10.1,10.4
1999-09-28 03:00:00,7.0,19.4
1999-09-28 12:00:00,6.3,14.4
...,...,...
2016-03-29 18:00:00,5.2,5.3
2016-03-30 00:00:00,-0.2,5.4
2016-03-30 06:00:00,-2.3,-2.8


In [354]:
df_three.loc['2001-01-01':'2002-01-01'].tail(30)

,kal,sar
time,,
2001-12-25 18:00:00,-15.8,-9.9
2001-12-25 21:00:00,-14.0,-13.9
2001-12-26 00:00:00,-17.3,-12.3
2001-12-26 03:00:00,-15.7,-13.0
2001-12-26 18:00:00,-4.9,-9.3
2001-12-26 21:00:00,-12.7,-15.4
2001-12-27 00:00:00,-16.9,-16.4
2001-12-27 03:00:00,-18.3,-19.8
2001-12-27 18:00:00,-19.6,-14.9


In [355]:
# how merge dataset with different indexes

df_sar.loc['2001-12-28':'2002-01-01 21:00:00']

,sar
time,
2001-12-28 00:00:00,-11.4
2001-12-28 03:00:00,-11.1
2001-12-28 09:00:00,-10.4
2001-12-28 15:00:00,-10.8
2001-12-28 18:00:00,-7.9
2001-12-28 21:00:00,-8.5
2001-12-29 00:00:00,-9.1
2001-12-29 03:00:00,-8.9
2001-12-29 09:00:00,-10.5


In [357]:
fdsfs = pd.concat([df_kal,df_sar, df_ser], axis=1)
fdsfs.loc['2001-12-28':'2002-01-01 21:00:00']

,T,Td,Tmax,Tmin,ddd,ff,prec,nt,nh,h,sar,ser
time,,,,,,,,,,,,
2001-12-28 00:00:00,-13.6,-17.9,-15.5,-----,SE,10.8,NaN,NaN,NaN,NaN,-11.4,-15.4
2001-12-28 03:00:00,-10.3,-12.3,-----,-----,SSE,18.0,NaN,NaN,NaN,NaN,-11.1,NaN
2001-12-28 06:00:00,-12.0,-13.3,-----,-----,SE,7.2,NaN,NaN,NaN,NaN,NaN,-11.8
2001-12-28 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10.4,NaN
2001-12-28 12:00:00,-12.1,-13.4,-----,-----,SSE,3.6,NaN,NaN,NaN,NaN,NaN,-11.4
2001-12-28 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10.8,NaN
2001-12-28 18:00:00,-10.6,-12.6,-----,-15.7,S,3.6,NaN,NaN,NaN,NaN,-7.9,-10.4
2001-12-28 21:00:00,-8.3,-9.7,-----,-----,ENE,21.6,NaN,NaN,NaN,NaN,-8.5,NaN
2001-12-29 00:00:00,-8.3,-9.3,-8.0,-----,ENE,14.4,NaN,NaN,NaN,NaN,-9.1,-7.6
